In [1]:
from gssutils import *

if is_interactive():
    scraper = Scraper('https://www.ons.gov.uk/businessindustryandtrade/internationaltrade/datasets/internationaltradeinservicesreferencetables')
    tabs = { tab.name: tab for tab in scraper.distribution().as_databaker() }

In [2]:
tab = tabs['Table C3 2013-2017']

In [3]:
observations = tab.excel_ref('E6').expand(DOWN).expand(RIGHT).is_not_blank()

In [4]:
Year = tab.excel_ref('E4').expand(RIGHT).is_not_whitespace()

In [5]:
Flow = tab.excel_ref('E3').expand(RIGHT).is_not_blank()

In [6]:
geo1 = tab.excel_ref('A5').expand(DOWN).is_not_blank() - tab.excel_ref('A94').expand(DOWN)

In [7]:
geo3 = tab.excel_ref('C5').expand(DOWN).is_not_blank() - tab.excel_ref('A94').expand(DOWN)

In [8]:
Dimensions = [
            HDim(Year,'Year',DIRECTLY,ABOVE),
            HDim(geo1,'geo1',CLOSEST,ABOVE),
#             HDim(geo2,'geo2',DIRECTLY,LEFT),
            HDim(geo3,'geo3',DIRECTLY,LEFT),
            HDim(Flow, 'Flow',CLOSEST,LEFT),
            HDimConst('Measure Type', 'GBP Total'),
            HDimConst('Unit','gbp-million')
            ]

In [9]:
c1 = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)
if is_interactive():
    savepreviewhtml(c1)

In [10]:
new_table = c1.topandas()

In [11]:
new_table['Year'] = pd.to_numeric(new_table['Year'], errors='coerce').fillna(0)

In [12]:
new_table['Year'] = new_table['Year'].astype(int)

In [13]:
new_table.columns = ['Value' if x=='OBS' else x for x in new_table.columns]

In [14]:
new_table['geo3'].fillna('Total', inplace = True)

In [15]:
new_table['BOP Service'] = new_table['geo1'].fillna('') + '-' + new_table['geo3'].fillna('')

In [16]:
new_table['Flow'] = new_table['Flow'].str.lower()

In [17]:
new_table['Flow'] = new_table['Flow'].map(lambda cell:cell.replace('balances', 'balance'))

In [18]:
new_table = new_table[new_table['Value'] != '' ]

In [19]:
new_table['International Trade Basis'] = 'BOP'

In [20]:
new_table['ONS Partner Geography'] = 'Whole world'

In [21]:
new_table['ITIS Industry'] = 'wholesale'

In [22]:
new_table = new_table[['ONS Partner Geography', 'Year','Flow','BOP Service','ITIS Industry', 'International Trade Basis', 'Measure Type','Value','Unit' ]]

In [23]:
# if is_interactive():
#     SubstancetinationFolder = Path('out')
#     SubstancetinationFolder.mkdir(exist_ok=True, parents=True)
#     new_table.to_csv(SubstancetinationFolder / ('tablec32.csv'), index = False)

In [24]:
new_table.head()

,ONS Partner Geography,Year,Flow,BOP Service,ITIS Industry,International Trade Basis,Measure Type,Value,Unit
4,Whole world,2017,exports,"Agricultural and Mining Services-Agricultural,...",wholesale,BOP,GBP Total,3,gbp-million
9,Whole world,2017,imports,"Agricultural and Mining Services-Agricultural,...",wholesale,BOP,GBP Total,5,gbp-million
10,Whole world,2013,balance,"Agricultural and Mining Services-Agricultural,...",wholesale,BOP,GBP Total,-4.29315,gbp-million
13,Whole world,2016,balance,"Agricultural and Mining Services-Agricultural,...",wholesale,BOP,GBP Total,2,gbp-million
14,Whole world,2017,balance,"Agricultural and Mining Services-Agricultural,...",wholesale,BOP,GBP Total,-2,gbp-million
